In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
from d2l import torch as d2l
#import d2lzh_pytorch as d2ll
import torch.nn.functional as F
import numpy as np
import torch
from torch import nn
from torch.nn import init

In [2]:
torch.cuda.is_available()

True

In [3]:
class ChannelAttention(nn.Module):
    def __init__(self,channel,reduction=16):
        super().__init__()
        self.maxpool=nn.AdaptiveMaxPool2d(1)
        self.avgpool=nn.AdaptiveAvgPool2d(1)
        self.se=nn.Sequential(
            nn.Conv2d(channel,channel//reduction,1,bias=False),
            nn.ReLU(),
            nn.Conv2d(channel//reduction,channel,1,bias=False)
        )
        self.sigmoid=nn.Sigmoid()
    
    def forward(self, x) :
        max_result=self.maxpool(x)
        avg_result=self.avgpool(x)
        max_out=self.se(max_result)
        avg_out=self.se(avg_result)
        output=self.sigmoid(max_out+avg_out)
        return output

class SpatialAttention(nn.Module):
    def __init__(self,kernel_size=7):
        super().__init__()
        self.conv=nn.Conv2d(2,1,kernel_size=kernel_size,padding=kernel_size//2)
        self.sigmoid=nn.Sigmoid()
    
    def forward(self, x) :
        max_result,_=torch.max(x,dim=1,keepdim=True)
        avg_result=torch.mean(x,dim=1,keepdim=True)
        result=torch.cat([max_result,avg_result],1)
        output=self.conv(result)
        output=self.sigmoid(output)
        return output


class CBAMBlock(nn.Module):

    def __init__(self, channel=512,reduction=16,kernel_size=7):
        super().__init__()
        self.ca=ChannelAttention(channel=channel,reduction=reduction)
        self.sa=SpatialAttention(kernel_size=kernel_size)


    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        b, c, _, _ = x.size()
        #residual=x
        out=x*self.ca(x)
        out=out*self.sa(out)
        return out#+residual


class cbamResidual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1,reduction=16):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.se =  CBAMBlock(num_channels, reduction)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.se(self.bn2(self.conv2(Y)))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
    
def bloodnet(num_classes, in_channels=1):
    
    def resnet_block(in_channels, out_channels, num_residuals,
                     first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(cbamResidual(in_channels, out_channels,
                                        use_1x1conv=True, strides=2))
            else:
                blk.append(cbamResidual(out_channels, out_channels))
        return nn.Sequential(*blk)

    
    net = nn.Sequential(
        nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU())
    net.add_module("resnet_block1", resnet_block(64, 64, 3, first_block=True))
    net.add_module("resnet_block2", resnet_block(64, 128, 4))
    net.add_module("resnet_block3", resnet_block(128, 256, 6))
    net.add_module("resnet_block4", resnet_block(256, 512, 3))
    net.add_module("global_avg_pool", nn.AdaptiveAvgPool2d((1,1)))
    net.add_module("fc", nn.Sequential(nn.Flatten(),
                                       nn.Linear(512, num_classes)))
    return net


def get_net():
    num_classes = 5
    net = bloodnet(num_classes, 3)
    return net

a=get_net()

In [4]:
a.load_state_dict(torch.load('./aaa/aaa/network_params.pth'))
model=a.eval()

In [5]:
model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (resnet_block1): Sequential(
    (0): cbamResidual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (se): CBAMBlock(
        (ca): ChannelAttention(
          (maxpool): AdaptiveMaxPool2d(output_size=1)
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (se): Sequential(
            (0): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): ReLU()
            (2): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (sigmoid): Sigmoid()

In [13]:
datadir="/home/omnisky/shenxiaochen/"
transform_test = torchvision.transforms.Compose([

    torchvision.transforms.CenterCrop(96),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

test_ds= torchvision.datasets.ImageFolder(
    os.path.join(datadir, 'test'),
    transform=transform_test)

batch_size=128
test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)

In [14]:
device=d2l.try_gpu()

In [15]:
# confusion matrix
model.to(device)
model.eval()
conf_mat=np.zeros((5,5))
for i,data in enumerate(test_iter):
    inputs, labels=data
    inputs, labels= inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs=model(inputs)
        _, predicted = torch.max(outputs.data,1)
        for j in range(len(labels)):
            cate_i=labels[j].cpu().numpy()
            pre_i =predicted[j].cpu().numpy()
            conf_mat[cate_i,pre_i]+=1

In [16]:
valid_confusematrix=pd.DataFrame(conf_mat)

In [17]:
valid_confusematrix

,0,1,2,3,4
0,1069.0,1.0,4.0,2.0,0.0
1,0.0,1147.0,0.0,0.0,1.0
2,9.0,0.0,938.0,15.0,0.0
3,2.0,0.0,21.0,987.0,0.0
4,1.0,1.0,0.0,0.0,1016.0


In [12]:
valid_confusematrix.to_csv("valid_confusematrix.csv")

In [15]:
conf_mat.trace()/conf_mat.sum()

0.9890678941311852

In [6]:
def evaluate_accuracy_gpu(net, data_iter, device=None): 
    
    if isinstance(net, torch.nn.Module):
        net.eval()  
        if not device:
            device = next(iter(net.parameters())).device
    
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            
            X = [x.to(device) for x in X]
        else:
            X = X.to(device)
        y = y.to(device)
        metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [7]:
# accuracy valid
device=d2l.try_gpu()
evaluate_accuracy_gpu(model.to(device),test_iter)

0.9890678941311852

In [8]:
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score,precision_score,recall_score,f1_score

In [9]:
preds=[]
true=[]
with torch.no_grad():
    for x,y in test_iter:
        x=x.to(device)
        y=y.to(device)
        output=model(x)
        output=output.argmax(dim=1)
        preds.extend(output.detach().cpu().numpy())
        true.extend(y.detach().cpu().numpy())
        
    

In [12]:
accuracy_score(true,preds)

0.9890678941311852

In [13]:
recall_score(true,preds,average='macro')

0.9885876809868922

In [14]:
f1_score(true,preds,average='macro')

0.9886191396291926

In [15]:
precision_score(true,preds,average='macro')

0.9886564856669147

In [16]:
from sklearn.preprocessing import label_binarize

In [17]:
test_trues=label_binarize(true,classes=[i for i in range(5)])

In [19]:
test_preds=label_binarize(preds,classes=[i for i in range(5)])

In [21]:
roc_auc_score(test_trues.ravel(),test_preds.ravel())# micro

0.9931674338319908

In [7]:
outsidetest_ds= torchvision.datasets.ImageFolder(
    os.path.join(datadir, 'outside_test'),
    transform=transform_test) 
outsidetest_iter = torch.utils.data.DataLoader(outsidetest_ds, batch_size, shuffle=False,
                                        drop_last=False)

In [11]:
# confusion matrix
model.to(device)
model.eval()
conf_mat=np.zeros((5,5))
for i,data in enumerate(outsidetest_iter):
    inputs, labels=data
    inputs, labels= inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs=model(inputs)
        _, predicted = torch.max(outputs.data,1)
        for j in range(len(labels)):
            cate_i=labels[j].cpu().numpy()
            pre_i =predicted[j].cpu().numpy()
            conf_mat[cate_i,pre_i]+=1

In [12]:
pd.DataFrame(conf_mat)

,0,1,2,3,4
0,5050.0,1.0,204.0,33.0,1.0
1,137.0,4404.0,331.0,6.0,349.0
2,12.0,1.0,3867.0,1142.0,2.0
3,16.0,1.0,525.0,4572.0,1.0
4,279.0,9.0,101.0,1.0,939.0


In [16]:
test_matrix=pd.DataFrame(conf_mat)

In [18]:
test_matrix.to_csv("test_confusematrix.csv")

In [19]:
conf_mat.trace()/conf_mat.sum()

0.8566229985443959

In [23]:
evaluate_accuracy_gpu(model.to(device),outsidetest_iter)

0.8566229985443959

In [24]:
o_preds=[]
o_true=[]
with torch.no_grad():
    for x,y in outsidetest_iter:
        x=x.to(device)
        y=y.to(device)
        output=model(x)
        output=output.argmax(dim=1)
        o_preds.extend(output.detach().cpu().numpy())
        o_true.extend(y.detach().cpu().numpy())

In [25]:
len(o_preds)

21984

In [26]:
len(o_true)

21984

In [27]:
accuracy_score(o_true,o_preds)

0.8566229985443959

In [31]:
test_trues=label_binarize(o_true,classes=[i for i in range(5)])
test_preds=label_binarize(o_preds,classes=[i for i in range(5)])

In [32]:
roc_auc_score(test_trues.ravel(),test_preds.ravel())

0.9103893740902473